<a href="https://colab.research.google.com/github/Mindauskas/iWild/blob/master/iWildcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook for iWildCam 2019 - FGVC6 image classification (https://www.kaggle.com/c/iwildcam-2019-fgvc6)

In [1]:
!pip install -q -U toai
!pip install -U kaggle

     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 296kB 8.3MB/s 
     |████████████████████████████████| 153kB 50.3MB/s 
     |████████████████████████████████| 61kB 13.5MB/s 
Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [0]:
def setup_kaggle():
  !mkdir /root/.kaggle
  !mv kaggle.json /root/.kaggle
  !chmod 600 /root/.kaggle/kaggle.json

In [0]:
setup_kaggle()

In [4]:
__import__('toai').__version__

'0.1.22'

In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [6]:
from toai.imports import *
from toai.data import Dataset, DataParams, DataContainer
from toai.models import save_keras_model, load_keras_model
from toai.metrics import sparse_top_2_categorical_accuracy
from toai.image import (
    ImageLearner,
    ImageAugmentor,
    ImageDataset,
    ImageParser,
    ImageResizer,
    LearningRateFinder,
    ImageTrainingStep,
    ImageTrainer,
)
from toai.utils import download_file, unzip, save_file, load_file
import tensorflow as tf
from tensorflow import keras

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
tf.enable_eager_execution()

**Download data from Kaggle**

In [8]:
os.getcwd()

'/content'

In [9]:
kaggle.api.authenticate()
kaggle.api.competition_download_files(
    competition='iwildcam-2019-fgvc6',
    path='competition_files',
    force=False,
    quiet=False)

  0%|          | 9.00M/43.4G [00:00<13:30, 57.5MB/s]

100%|██████████| 43.4G/43.4G [08:04<00:00, 96.3MB/s]

In [10]:
%cd competition_files/

/content/competition_files


In [11]:
!unzip iwildcam-2019-fgvc6.zip

Archive:  iwildcam-2019-fgvc6.zip
  inflating: train_images.zip        
  inflating: train.csv               
  inflating: test_images.zip         
  inflating: sample_submission.csv   
  inflating: test.csv                


In [0]:
!rm iwildcam-2019-fgvc6.zip

In [0]:
!mkdir train
!unzip -qq train_images.zip -d train
!rm train_images.zip

In [0]:
!mkdir test
!unzip -qq test_images.zip -d test
!rm test_images.zip

In [0]:
train_labels = pd.read_csv('train.csv')

In [16]:
train_labels.shape

(196299, 11)

In [17]:
train_labels.head(5)

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
0,19,2011-05-13 23:43:18,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,1,5998cfa4-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,6f084ccc-5567-11e8-bc84-dca9047ef277,3,1024,747
1,19,2012-03-17 03:48:44,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,2,588a679f-23d2-11e8-a6a3-ec086b02610b,115,Justin Brown,6f12067d-5567-11e8-b3c0-dca9047ef277,3,1024,747
2,0,2014-05-11 11:56:46,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,1,59279ce3-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6faa92d1-5567-11e8-b1ae-dca9047ef277,1,1024,747
3,0,2013-10-06 02:00:00,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,1,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,57,Erin Boydston,6f7d4702-5567-11e8-9e03-dca9047ef277,1,1024,747
4,0,2011-07-12 13:11:16,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg,3,599fbd89-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f1728a1-5567-11e8-9be7-dca9047ef277,3,1024,747


In [18]:
train_labels.category_id.value_counts(dropna=False)

0     131457
19     14106
13      8623
11      7209
8       6938
1       6102
16      5975
17      4759
3       3398
18      3035
4       2210
14      1361
10      1093
22        33
Name: category_id, dtype: int64

Some labels do not exist in training dataset (e.g. 2,5, etc.)!

**Creating folder for each category**

In [0]:
!mkdir train_data_in_folders

In [0]:
def make_dir(target):
  if not os.path.exists(target):
    os.mkdir(target)
  return target

In [0]:
def split_to_folders(train_labels_df, source, destination):
  dir_data = Path(source)
  for _, row in train_labels_df.iterrows():
    file_name = row.file_name
    category_id = row.category_id
    target_dest = make_dir(Path(destination)/str(category_id))
    try:
      shutil.move('./'+source+'/'+file_name, target_dest)
    except Exception as e:
      print(e)

In [22]:
split_to_folders(train_labels,'train','train_data_in_folders')

Destination path 'train_data_in_folders/16/5858c2ed-23d2-11e8-a6a3-ec086b02610b.jpg' already exists
[Errno 2] No such file or directory: './train/5858c2ed-23d2-11e8-a6a3-ec086b02610b.jpg'
[Errno 2] No such file or directory: './train/5858c2ed-23d2-11e8-a6a3-ec086b02610b.jpg'
Destination path 'train_data_in_folders/16/5919977e-23d2-11e8-a6a3-ec086b02610b.jpg' already exists
[Errno 2] No such file or directory: './train/5919977e-23d2-11e8-a6a3-ec086b02610b.jpg'
[Errno 2] No such file or directory: './train/5919977e-23d2-11e8-a6a3-ec086b02610b.jpg'
Destination path 'train_data_in_folders/16/5892b57f-23d2-11e8-a6a3-ec086b02610b.jpg' already exists
[Errno 2] No such file or directory: './train/5892b57f-23d2-11e8-a6a3-ec086b02610b.jpg'
[Errno 2] No such file or directory: './train/5892b57f-23d2-11e8-a6a3-ec086b02610b.jpg'
Destination path 'train_data_in_folders/16/58fd7e46-23d2-11e8-a6a3-ec086b02610b.jpg' already exists
[Errno 2] No such file or directory: './train/58fd7e46-23d2-11e8-a6a3-ec

**Load data from subfolders**

In [23]:
train_labels[train_labels.file_name == '58d930f0-23d2-11e8-a6a3-ec086b02610b.jpg']

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
54243,8,2012-05-23 05:24:12,58d930f0-23d2-11e8-a6a3-ec086b02610b.jpg,2,58d930f0-23d2-11e8-a6a3-ec086b02610b,130,Justin Brown,6f0d87d1-5567-11e8-8daa-dca9047ef277,3,1024,747
54244,8,2012-05-23 05:24:12,58d930f0-23d2-11e8-a6a3-ec086b02610b.jpg,2,58d930f0-23d2-11e8-a6a3-ec086b02610b,130,Justin Brown,6f0d87d1-5567-11e8-8daa-dca9047ef277,3,1024,747
54245,16,2012-05-23 05:24:12,58d930f0-23d2-11e8-a6a3-ec086b02610b.jpg,2,58d930f0-23d2-11e8-a6a3-ec086b02610b,130,Justin Brown,6f0d87d1-5567-11e8-8daa-dca9047ef277,3,1024,747
54246,16,2012-05-23 05:24:12,58d930f0-23d2-11e8-a6a3-ec086b02610b.jpg,2,58d930f0-23d2-11e8-a6a3-ec086b02610b,130,Justin Brown,6f0d87d1-5567-11e8-8daa-dca9047ef277,3,1024,747


**Model**

In [0]:
IMG_DIMS = (100, 100, 3)

In [0]:
train_data, valid_data, test_data = ImageDataset.split(
    dataset=ImageDataset.from_subfolders('./train_data_in_folders/'),
    fracs=(0.8, 0.1, 0.1),
)

In [26]:
!pwd

/content/competition_files


In [0]:
train_data = (
    train_data
    .dataset(batch_size=32, img_dims=IMG_DIMS, shuffle=True)
    .make_pipeline(
        image_pipeline=[
            ImageParser(),
            ImageResizer(img_dims=IMG_DIMS, resize="crop", crop_adjustment=1),
            ImageAugmentor(level=3, flips="both"),
        ],
    )
    .save_pipeline('./train_files')
    .preprocess()
)

In [0]:
valid_data = (
    valid_data
    .dataset(batch_size=1, img_dims=IMG_DIMS, shuffle=False)
    .make_pipeline(
        label_map=train_data.label_map,
        image_pipeline=[
            ImageParser(),
            ImageResizer(img_dims=IMG_DIMS, resize="stretch"),
        ],
    )
    .save_pipeline('./valid_files')
    .preprocess()
)

In [0]:
test_data = (
    test_data
    .dataset(batch_size=1, img_dims=IMG_DIMS, shuffle=False)
    .load_pipeline('./valid_files')
    .preprocess()
)

In [30]:
train_data.label_map

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '13': 4,
 '14': 5,
 '16': 6,
 '17': 7,
 '18': 8,
 '19': 9,
 '22': 10,
 '3': 11,
 '4': 12,
 '8': 13}

In [31]:
valid_data.label_map

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '13': 4,
 '14': 5,
 '16': 6,
 '17': 7,
 '18': 8,
 '19': 9,
 '22': 10,
 '3': 11,
 '4': 12,
 '8': 13}

In [32]:
test_data.label_map

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '13': 4,
 '14': 5,
 '16': 6,
 '17': 7,
 '18': 8,
 '19': 9,
 '22': 10,
 '3': 11,
 '4': 12,
 '8': 13}

In [33]:
len(train_data), len(valid_data), len(test_data)

(156869, 19609, 19608)

In [0]:
data_container = DataContainer(
    train=train_data,
    validation=valid_data,
    test=test_data,
)

In [0]:
#data_container.train.show(debug=True)

In [0]:
#data_container.validation.show()

In [0]:
#data_container.test.show()

In [38]:
learner_xception = ImageLearner(
    path='./Xception',
    base_model=keras.applications.Xception,
    input_shape=(IMG_DIMS),
    output_shape=[data_container.train.n_classes],
    activation=keras.activations.softmax,
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=[keras.metrics.sparse_categorical_accuracy, sparse_top_2_categorical_accuracy],
    dropout=0.5,
    l1=3e-6,
    l2=3e-5,
    load=True,
)

83689472/83683744 [==============================] - 1s 0us/step


In [0]:
trainer = ImageTrainer(
    learner=learner_xception,
    data_container=data_container,
    steps=[
            ImageTrainingStep(
        n_epochs=10,
        lr=3e-5,
        optimizer=keras.optimizers.Adam,
        feature_pipeline=[
            ImageParser(),
            ImageResizer(img_dims=IMG_DIMS, resize="stretch"),
            ImageAugmentor(level=1),
        ],
    ),
    ],
)

In [40]:
trainer.train()

Train for 4903 steps, validate for 19609 steps
Epoch 1/10
4903/4903 [==============================] - 2516s 513ms/step - loss: 0.8933 - sparse_categorical_accuracy: 0.7746 - sparse_top_2_categorical_accuracy: 0.8450 - val_loss: 0.4566 - val_sparse_categorical_accuracy: 0.8638 - val_sparse_top_2_categorical_accuracy: 0.9265
Epoch 2/10
4903/4903 [==============================] - 1668s 340ms/step - loss: 0.4223 - sparse_categorical_accuracy: 0.8713 - sparse_top_2_categorical_accuracy: 0.9343 - val_loss: 0.7374 - val_sparse_categorical_accuracy: 0.8575 - val_sparse_top_2_categorical_accuracy: 0.9170
Epoch 3/10
4903/4903 [==============================] - 1678s 342ms/step - loss: 0.2847 - sparse_categorical_accuracy: 0.9089 - sparse_top_2_categorical_accuracy: 0.9611 - val_loss: 0.3604 - val_sparse_categorical_accuracy: 0.8947 - val_sparse_top_2_categorical_accuracy: 0.9483
Epoch 4/10
4903/4903 [==============================] - 1667s 340ms/step - loss: 0.2120 - sparse_categorical_accurac

In [41]:
trainer.evaluate_dataset()

19609/19609 [==============================] - 336s 17ms/step - loss: 0.2496 - sparse_categorical_accuracy: 0.9255 - sparse_top_2_categorical_accuracy: 0.9705


[0.2496061964247681, 0.9254934, 0.9705237]

In [42]:
trainer.predict_dataset()

array([[1.9806159e-01, 2.6672569e-04, 1.2769988e-04, ..., 3.0102677e-04,
        1.0660567e-03, 1.0610427e-03],
       [1.3429603e-04, 2.2704693e-04, 2.8346459e-04, ..., 2.2895604e-04,
        1.7817719e-03, 4.4818101e-03],
       [9.9989545e-01, 5.4816219e-06, 4.1931039e-06, ..., 2.2651089e-05,
        1.1633425e-05, 2.3497027e-05],
       ...,
       [9.9980825e-01, 3.0459900e-05, 1.5767733e-05, ..., 3.2395314e-05,
        2.8783215e-05, 1.7015866e-05],
       [4.1828245e-02, 4.1348166e-03, 1.8093910e-03, ..., 1.6297715e-03,
        3.9462787e-03, 1.8624712e-02],
       [6.5587187e-04, 1.8090500e-04, 2.1232374e-04, ..., 3.0742399e-04,
        1.1657493e-04, 1.2717665e-04]], dtype=float32)

In [43]:
trainer.analyse_dataset()

,path,image,label,label_code,pred,pred_code,label_probs,pred_probs
0,train_data_in_folders/19/5a1b1528-23d2-11e8-a6...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",19,9,19,9,0.537715,0.537715
1,train_data_in_folders/11/59941e4d-23d2-11e8-a6...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",11,3,13,4,0.003452,0.942632
2,train_data_in_folders/0/589609ef-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",0,0,0,0,0.999895,0.999895
3,train_data_in_folders/0/58e40ecf-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",0,0,0,0,0.988748,0.988748
4,train_data_in_folders/0/596a4eef-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",0,0,0,0,0.999294,0.999294
5,train_data_in_folders/0/58e598f5-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",0,0,0,0,0.999887,0.999887
6,train_data_in_folders/19/586fdd79-23d2-11e8-a6...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",19,9,19,9,0.998742,0.998742
7,train_data_in_folders/8/595addc1-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0793725...",8,13,4,12,0.000176,0.983095
8,train_data_in_folders/0/58c66444-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0793725...",0,0,0,0,0.999788,0.999788
9,train_data_in_folders/0/59fae537-23d2-11e8-a6a...,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0715293...",0,0,0,0,0.999516,0.999516


In [0]:
trainer.show_predictions()